# 1. Data Colletion

For my project, I collected data from a company called Robinhood, which specializes in offering its users the ability to invest in stocks without having to pay any commissions on the transactions. On their website, they have collections of stocks listed with various information about each one, including the stock ticker, price, analyst ratings, etc. I downloaded the html files of each of these collections and as you can see below, I read them all in along with all of the information that Robinhood provided.

In [1]:
%matplotlib inline
import codecs
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

files = ["Consumer", "Energy", "Entertainment", "Finance", "Health",
        "Manufacturing", "Oil:Gas", "SocialMedia", "Software", "Tech"]
rows = []

for file in files:
    filename = file + ".htm"
    f = codecs.open(filename, 'r')
    soup = BeautifulSoup(f.read(), "html.parser")
    tables = soup.find_all("table")
    stock_data = tables[0]
    
    for stock in stock_data.find_all("tr", {"class":"_3-Fg9lFlzey28mCJClXXxZ"}):
        name = stock.find("div", {"class":"_2fMBL180hIqVoxOuNVJgST"}).text
        symbol = stock.find("span").text
        price = stock.find_all("div")[4].text
        market_cap = stock.find_all("div")[7].text
        owners = stock.find("div", {"class":"OJSR51UkZBkOak0WhbWKn"}).text
        rating = stock.find_all("div")[10].text
        rows.append({
        "Company": name,
        "Ticker": symbol,
        "Price": price, 
        "MarketCap": market_cap,
        "Popularity": owners,
        "Rating": rating,
        "Sector": file
        })
    
df = pd.DataFrame(rows)

I set the stock ticker as the index since this should be unique to each stock, and then I transformed the quantitative variables to their correct forms. The Market Cap of a stock was given in millions or billions with corresponding "M"'s and "B"'s at the end of the number, so I had to convert these to numeric as well. Then I made a bivariate variable for each of the possible sectors to determine if a stock is classified under a specific sector, and a stock can be classified under multiple sectors.

In [2]:
rh = df.set_index("Ticker").replace("—", np.nan)
rh["Popularity"] = rh['Popularity'].astype(str
            ).str.replace(",", "").astype(float)
rh["Price"] = rh["Price"].astype(str).str.replace("$", ""
                    ).str.replace(",", "").astype(float)

def mc_to_float(mc):
    billion = True
    if "M" in mc:
        billion = False
    if billion:
        return float(mc.replace("B", "")) * 1000000
    else:
        return float(mc.replace("M", "")) * 1000
    
rh["MarketCap"] = rh["MarketCap"].astype(str).apply(mc_to_float)
rh["Rating"] = pd.to_numeric(rh["Rating"].str.replace("% Buy",""))

sectors = pd.get_dummies(rh["Sector"]).groupby(rh.index).sum()
rh = rh.drop_duplicates().merge(sectors, how="inner", left_index=True,
        right_index=True).drop("Sector", axis=1).drop_duplicates()

MarketWatch.com displays the change in price of a stock for various intervals, so for each stock, I retrieved the change in price over the past five days, month, three months, and year. I also got the volume traded per day as well.

In [ ]:
import requests
import time

resp = requests.get("https://www.marketwatch.com/investing/stock/aapl")
soup = BeautifulSoup(resp.content, "html.parser")

five_day = []
month = []
three_month = []
year = []
volumes = []

for ticker in rh.index:
    resp = requests.get("https://www.marketwatch.com/investing/stock/" + ticker)
    soup = BeautifulSoup(resp.content, "html.parser")
    changes = soup.findAll("li", {"class":"content__item value ignore-color"})
    print(ticker)
    if (len(changes) > 1):
        five_day.append(changes[0].text)
        month.append(changes[1].text)
    else:
        five_day.append("NA")
        month.append("NA")
    if (len(changes) > 4):
        three_month.append(changes[2].text)
        year.append(changes[4].text) 
    else:
        three_month.append("NA")
        year.append("NA")
    if len(soup.findAll("span", {"class":"kv__primary"})) > 0:
        volumes.append(soup.findAll("span", {"class":"kv__primary"})[-1].text)
    else:
        volumes.append("NA")
    time.sleep(.2)

In [4]:
rh["Year"] = year
rh["Three_Month"] = three_month
rh["Month"] = month
rh["Five_Day"] = five_day
rh["Year"] = pd.to_numeric(rh["Year"].str.replace("%",""), errors="coerce")
rh["Month"] = pd.to_numeric(rh["Month"].str.replace("%",""), errors="coerce")
rh["Three_Month"] = pd.to_numeric(rh["Three_Month"].str.replace("%",""), errors="coerce")
rh["Five_Day"] = pd.to_numeric(rh["Five_Day"].str.replace("%",""), errors="coerce")
rh["Volume"] = volumes

def transform_volume(volume):
    if "M" in volume:
        return float(volume.replace("M", "")) * 1000000
    if "K" in volume:
        return float(volume.replace("K", "")) * 1000
    return float(volume)

rh["Volume"] = rh["Volume"].apply(transform_volume)

Yahoo Finance gives a description of each company based off of it's ticker, so I was also able to get this textual data for each individual stock. I was able to use these company descriptions to create some variables which stated whether or not the given stock's description contained a specified keyword.

In [5]:
descriptions = []

for ticker in rh.index:
    resp = requests.get("https://finance.yahoo.com/quote/%s/profile?p=%s"%(ticker,ticker))
    soup = BeautifulSoup(resp.content, "html.parser")
    desc = soup.findAll("p", {"class":"Mt(15px) Lh(1.6)"})
    if len(desc) > 0:
        descriptions.append(desc[0].text)
    else:
        descriptions.append("NA")
    time.sleep(.1)
    
rh["Description"] = descriptions

keywords = ["machine learning", "artificial intelligence",
            "robotics", "gaming", "software", "solar", "cannabis"]

for keyword in keywords:
    rh[keyword] = rh.Description.str.lower().str.contains(keyword)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


I found out that MarketWatch also gives the location of the headquarters of each company, so I was able to get the address of each one.

In [ ]:
rh.to_csv("temp.csv")

addresses = []
for ticker in rh.index:
    resp = requests.get("https://www.marketwatch.com/investing/stock/%s/profile"%ticker)
    soup = BeautifulSoup(resp.content, "html.parser")
    addy = soup.findAll("div", {"companyinfo fourwide"})
    if len(addy) > 0:
        addresses.append(addy[0])
    else:
        addresses.append("NA")
    time.sleep(.1)
    print(ticker)

In [7]:
final_add = []

for address in addresses:
    if len(address) > 4:
        final_add.append(address.text.split("\n")[2] + " " + address.text.split("\n")[-3])
    else:
        final_add.append("NA")
        
rh["Address"] = final_add
rh.to_csv("temp2.csv")

I then found the geopy library, which allowed me to obtain a set of coordinates (longitude, latitude) for each of the addresses that I found above.

In [ ]:
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

rh = pd.read_csv("temp2.csv")

longs = []
lats = []

count = 0
for addy in rh.Address:
    print(count)
    geolocator = Nominatim(user_agent="project")
    location = geolocator.geocode(addy)
    if location != None:
        longs.append(location.longitude)
        lats.append(location.latitude)
    else:
        longs.append("NA")
        lats.append("NA")
    count += 1

rh["Longitude"] = longs
rh["Latitude"] = lats

Then, I wanted to get some info on the CEO of each company, so I used MarketWatch to get the age and name of the main director in charge of the company.

In [ ]:
rh.to_csv("temp3.csv")

names = []
ages = []

for ticker in rh.index:
    resp = requests.get("https://www.marketwatch.com/investing/stock/%s/profile"%ticker)
    soup = BeautifulSoup(resp.content, "html.parser")
    if len(soup.findAll("td", {"class":"name"})) > 0:
        names.append(soup.findAll("td", {"class":"name"})[0].text)
    else:
        names.append("NA")
    if len(soup.findAll("td", {"class":"age"})) > 0:
        ages.append(soup.findAll("td", {"class":"age"})[0].text)
    else:
        ages.append("NA")
    print(ticker)

Then I used the prefix of the CEO's names to determine the sex of the CEO.

In [38]:
rh["CEO_name"] = names
rh["CEO_age"] = pd.to_numeric(ages, errors="coerce")
rh["CEO_sex"] = rh["CEO_name"].str.split().str[0].map({
    "Mr.":0,
    "NA":None,
    "Dr.":None,
    "Ms.":1,
    "Howard":0,
    "Raj":0,
    "Gary":0,
    "GCP":None,
    "John":0,
    "Scott":0,
    "Chesapeake": None,
    "David":0,
    "Jennifer":1,
    "Michael":0
})

rh["CEO_age"] = pd.to_numeric(rh["CEO_age"], errors="coerce")



Lastly, I looked at whether or not the stock ticker was an English word or not, so that I could later see if this influenced user's decisions on which stocks were the most appealing.

In [65]:
import enchant
d = enchant.Dict("en_US")
rh["English_Word"] = rh["Ticker"].apply(d.check)

In [68]:
rh.to_csv("final.csv")